In [ ]:
#@title 🎧 Download Narration Audio & Play Introduction
import os as _os
if not _os.path.exists("/content/narration"):
    !pip install -q gdown
    import gdown
    gdown.download(id="17rFuCNZUUY1xHrMq1WTamV-JWh_IDZe8", output="/content/narration.zip", quiet=False)
    !unzip -q /content/narration.zip -d /content/narration
    !rm /content/narration.zip
    print(f"Loaded {len(_os.listdir('/content/narration'))} narration segments")
else:
    print("Narration audio already loaded.")

from IPython.display import Audio, display
display(Audio("/content/narration/04_00_intro.mp3"))


In [ ]:
#@title 🎧 Code Walkthrough: Layernorm Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_06_layernorm_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

In [ ]:
#@title 🎧 Listen: Why It Matters
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_02_why_it_matters.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


# The Full Transformer Block: From Attention to Architecture

*Part 4 of the Vizuara series on Self-Attention from First Principles*
*Estimated time: 55 minutes*

## 1. Why Does This Matter?

Over the last three notebooks, we have built every component of self-attention from the ground up: QKV projections, scaled dot-product attention, multi-head attention, and positional encoding. Now it is time to assemble these pieces into the **Transformer block** — the fundamental building block of GPT, BERT, LLaMA, and every other modern language model.

A single Transformer block adds two critical ingredients to our attention mechanism: **residual connections** (skip connections) and **layer normalization**. Together with a **feed-forward network**, these form a complete Transformer layer. Stack 6, 12, or 96 of these blocks, and you have a state-of-the-art language model.

By the end of this notebook, you will:
- Build a complete Transformer encoder block from scratch
- Train it on a real text classification task (IMDB sentiment)
- Visualize attention patterns on actual English sentences
- Understand why residual connections and layer norm are essential

In [ ]:
#@title 🎧 Listen: Building Intuition
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_03_building_intuition.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 2. Building Intuition

Think of building a Transformer block like constructing a building floor:

1. **Multi-Head Attention** is the main conference room where everyone shares information with everyone else.
2. **Residual Connection** is the elevator shaft — it lets information bypass the conference room and flow directly to the next floor. Even if the conference room adds nothing useful, the original information is preserved.
3. **Layer Normalization** is the HVAC system — it keeps the "temperature" (scale of activations) consistent across all rooms on the floor.
4. **Feed-Forward Network** is the private offices where each person processes information independently after the group discussion.

Stack these floors, and each floor refines the representations further.

### Think About This

What would happen if we removed the residual connections? Without them, every piece of information must pass *through* the attention and FFN layers — if those layers are poorly initialized, the signal gets corrupted. The residual connection guarantees that at worst, the block is the identity function — it cannot make things worse.

In [ ]:
#@title 🎧 Listen: The Mathematics
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_04_the_mathematics.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 3. The Mathematics

A single Transformer encoder block computes:

**Sub-layer 1: Multi-Head Attention + Add & Norm**

$$\text{sublayer}_1 = \text{LayerNorm}(x + \text{MultiHead}(x))$$

**Sub-layer 2: Feed-Forward Network + Add & Norm**

$$\text{sublayer}_2 = \text{LayerNorm}(\text{sublayer}_1 + \text{FFN}(\text{sublayer}_1))$$

where the FFN is:

$$\text{FFN}(x) = \max(0, \; x W_1 + b_1) \, W_2 + b_2$$

**What this says computationally:** Take the input $x$, run it through multi-head attention, then add the original $x$ back (residual connection), and normalize. Feed the result through a two-layer neural network (with ReLU activation), add another residual connection, and normalize again. The output has the same shape as the input — $(n, d_{\text{model}})$ — so blocks can be stacked.

The FFN typically has a hidden dimension of $4 \times d_{\text{model}}$. In the original Transformer with $d_{\text{model}} = 512$, the FFN hidden size is 2048.

In [ ]:
#@title 🎧 Transition: Layernorm Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_05_layernorm_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 4. Let's Build It — Component by Component

### 4.1 Layer Normalization

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader, TensorDataset

# First, let's understand what Layer Normalization does
class LayerNorm(nn.Module):
    """Layer Normalization implemented from scratch."""
    def __init__(self, d_model, eps=1e-6):
        super().__init__()
        self.gamma = nn.Parameter(torch.ones(d_model))   # Learnable scale
        self.beta = nn.Parameter(torch.zeros(d_model))    # Learnable shift
        self.eps = eps

    def forward(self, x):
        # Compute mean and variance along the last dimension
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)

        # Normalize
        x_norm = (x - mean) / torch.sqrt(var + self.eps)

        # Scale and shift
        return self.gamma * x_norm + self.beta

# Compare our implementation with PyTorch's
torch.manual_seed(42)
x_test = torch.randn(2, 5, 16)

our_ln = LayerNorm(16)
pytorch_ln = nn.LayerNorm(16)

out_ours = our_ln(x_test)
out_pytorch = pytorch_ln(x_test)

print(f"Our LayerNorm output mean (should be ~0): {out_ours.mean(dim=-1)[0].tolist()}")
print(f"Our LayerNorm output std (should be ~1):  {out_ours.std(dim=-1)[0].tolist()}")
print(f"Outputs match: {torch.allclose(out_ours, out_pytorch, atol=1e-5)}")

In [ ]:
#@title 🎧 What to Look For: Layernorm Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_07_layernorm_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Visualize what Layer Norm does
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

x_vis = torch.randn(1, 8, 16) * 5 + 3  # Large scale, shifted mean
x_normed = our_ln(x_vis)

# Before normalization
ax = axes[0]
im = ax.imshow(x_vis[0].detach().numpy(), cmap='RdBu_r', aspect='auto', vmin=-15, vmax=15)
ax.set_title('Before LayerNorm', fontsize=12, fontweight='bold')
ax.set_xlabel('Dimension')
ax.set_ylabel('Position')
plt.colorbar(im, ax=ax, shrink=0.8)

# After normalization
ax = axes[1]
im = ax.imshow(x_normed[0].detach().numpy(), cmap='RdBu_r', aspect='auto', vmin=-3, vmax=3)
ax.set_title('After LayerNorm', fontsize=12, fontweight='bold')
ax.set_xlabel('Dimension')
ax.set_ylabel('Position')
plt.colorbar(im, ax=ax, shrink=0.8)

# Distribution comparison
ax = axes[2]
ax.hist(x_vis[0].detach().numpy().flatten(), bins=30, alpha=0.7, label='Before', color='#e74c3c')
ax.hist(x_normed[0].detach().numpy().flatten(), bins=30, alpha=0.7, label='After', color='#3498db')
ax.set_title('Value Distribution', fontsize=12, fontweight='bold')
ax.legend()
ax.set_xlabel('Value')

plt.suptitle('Layer Normalization: Standardizing Activations', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 Listen: Ffn Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_08_ffn_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.2 Feed-Forward Network

In [ ]:
#@title 🎧 Code Walkthrough: Ffn Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_09_ffn_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
class FeedForwardNetwork(nn.Module):
    """Position-wise Feed-Forward Network."""
    def __init__(self, d_model, d_ff, dropout=0.1):
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # x: (batch, seq, d_model)
        x = self.linear1(x)       # (batch, seq, d_ff)
        x = F.relu(x)             # ReLU activation
        x = self.dropout(x)
        x = self.linear2(x)       # (batch, seq, d_model)
        return x

# Test
ffn = FeedForwardNetwork(d_model=32, d_ff=128)
x_ffn = torch.randn(1, 6, 32)
out_ffn = ffn(x_ffn)
print(f"FFN input shape:  {x_ffn.shape}")
print(f"FFN output shape: {out_ffn.shape}")
print(f"Expansion ratio: {128/32}x (d_ff / d_model)")

In [ ]:
#@title 🎧 Listen: Mha Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_10_mha_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.3 Multi-Head Self-Attention (from previous notebook)

In [ ]:
#@title 🎧 Code Walkthrough: Mha Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_11_mha_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, d_model, num_heads, dropout=0.1):
        super().__init__()
        assert d_model % num_heads == 0
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        self.W_q = nn.Linear(d_model, d_model, bias=False)
        self.W_k = nn.Linear(d_model, d_model, bias=False)
        self.W_v = nn.Linear(d_model, d_model, bias=False)
        self.W_o = nn.Linear(d_model, d_model, bias=False)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        B, T, C = x.size()
        Q = self.W_q(x).view(B, T, self.num_heads, self.d_k).transpose(1, 2)
        K = self.W_k(x).view(B, T, self.num_heads, self.d_k).transpose(1, 2)
        V = self.W_v(x).view(B, T, self.num_heads, self.d_k).transpose(1, 2)

        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            scores = scores.masked_fill(~mask, float('-inf'))
        attn_weights = self.dropout(F.softmax(scores, dim=-1))
        context = torch.matmul(attn_weights, V)

        context = context.transpose(1, 2).contiguous().view(B, T, self.d_model)
        return self.W_o(context), attn_weights

In [ ]:
#@title 🎧 Transition: Block Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_12_block_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.4 The Complete Transformer Block

In [ ]:
#@title 🎧 Code Walkthrough: Block Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_13_block_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
class SinusoidalPositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float32).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2, dtype=torch.float32) * (-math.log(10000.0) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, :x.size(1), :]


class TransformerBlock(nn.Module):
    """
    A single Transformer encoder block.
    Architecture: MultiHeadAttention -> Add & Norm -> FFN -> Add & Norm
    """
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        self.attention = MultiHeadSelfAttention(d_model, num_heads, dropout)
        self.ffn = FeedForwardNetwork(d_model, d_ff, dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        # Sub-layer 1: Multi-Head Attention + Residual + Norm
        attn_output, attn_weights = self.attention(x, mask)
        x = self.norm1(x + self.dropout1(attn_output))  # Residual connection

        # Sub-layer 2: FFN + Residual + Norm
        ffn_output = self.ffn(x)
        x = self.norm2(x + self.dropout2(ffn_output))    # Residual connection

        return x, attn_weights

# Test the complete block
torch.manual_seed(42)
block = TransformerBlock(d_model=32, num_heads=4, d_ff=128)
x_block = torch.randn(2, 6, 32)
out_block, attn_block = block(x_block)

print(f"Input shape:            {x_block.shape}")
print(f"Output shape:           {out_block.shape}")
print(f"Attention weights shape: {attn_block.shape}")
print(f"\nInput and output have the same shape — blocks can be stacked!")

In [ ]:
#@title 🎧 What to Look For: Residual Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_14_residual_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Demonstrate that residual connections preserve information
# Compare block output with and without residuals

class TransformerBlockNoResidual(nn.Module):
    """Transformer block WITHOUT residual connections (for comparison)."""
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        self.attention = MultiHeadSelfAttention(d_model, num_heads, dropout)
        self.ffn = FeedForwardNetwork(d_model, d_ff, dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)

    def forward(self, x, mask=None):
        attn_output, _ = self.attention(x, mask)
        x = self.norm1(attn_output)  # NO residual
        ffn_output = self.ffn(x)
        x = self.norm2(ffn_output)   # NO residual
        return x

# Stack 10 blocks and see how signal propagates
torch.manual_seed(42)
d_model_demo = 32
x_input = torch.randn(1, 6, d_model_demo)

# With residuals
norms_with_res = [x_input.norm().item()]
x_with = x_input.clone()
for _ in range(10):
    block_r = TransformerBlock(d_model_demo, 4, 128, dropout=0.0)
    block_r.eval()
    x_with, _ = block_r(x_with)
    norms_with_res.append(x_with.norm().item())

# Without residuals
norms_without_res = [x_input.norm().item()]
x_without = x_input.clone()
for _ in range(10):
    block_nr = TransformerBlockNoResidual(d_model_demo, 4, 128, dropout=0.0)
    block_nr.eval()
    x_without = block_nr(x_without)
    norms_without_res.append(x_without.norm().item())

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(range(11), norms_with_res, 'o-', label='With Residuals', color='#2ecc71', linewidth=2, markersize=6)
ax.plot(range(11), norms_without_res, 's-', label='Without Residuals', color='#e74c3c', linewidth=2, markersize=6)
ax.set_xlabel('Layer Depth', fontsize=12)
ax.set_ylabel('Activation Norm', fontsize=12)
ax.set_title('Signal Propagation: With vs Without Residual Connections', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()
print("With residuals: activation norms stay stable.")
print("Without residuals: activations may collapse or explode.")

In [ ]:
#@title 🎧 Before You Start: Todo1 Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_15_todo1_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 5. Your Turn

### TODO 1: Build a Full Transformer Encoder

In [ ]:
#@title 🎧 Before You Start: Todo1 Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_16_todo1_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
class TransformerEncoder(nn.Module):
    """
    Full Transformer encoder: Embedding + PE + N stacked blocks.

    Args:
        vocab_size: Number of tokens in the vocabulary
        d_model: Model dimension
        num_heads: Number of attention heads per block
        d_ff: Feed-forward hidden dimension
        num_layers: Number of Transformer blocks to stack
        max_len: Maximum sequence length
        dropout: Dropout rate
    """
    def __init__(self, vocab_size, d_model, num_heads, d_ff, num_layers, max_len=512, dropout=0.1):
        super().__init__()
        # ============ TODO ============
        # 1. Create a token embedding layer: nn.Embedding(vocab_size, d_model)
        # 2. Create a positional encoding: SinusoidalPositionalEncoding(d_model, max_len)
        # 3. Create a list of TransformerBlock layers: nn.ModuleList
        # 4. Create a dropout layer
        # ==============================

        self.embedding = ???      # YOUR CODE HERE
        self.pos_encoding = ???   # YOUR CODE HERE
        self.blocks = ???         # YOUR CODE HERE
        self.dropout = ???        # YOUR CODE HERE
        self.d_model = d_model

    def forward(self, x, mask=None):
        """
        Args:
            x: Token IDs of shape (batch_size, seq_len)
            mask: Optional attention mask

        Returns:
            output: Shape (batch_size, seq_len, d_model)
            all_attention_weights: List of attention weights from each layer
        """
        # ============ TODO ============
        # 1. Embed tokens and scale by sqrt(d_model)
        # 2. Add positional encoding
        # 3. Apply dropout
        # 4. Pass through each Transformer block, collecting attention weights
        # ==============================

        all_attn = []

        # YOUR CODE HERE

        return x, all_attn

In [ ]:
#@title 🎧 Code Walkthrough: Todo1 After
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_17_todo1_after.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 Code Walkthrough: Todo2 After
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_20_todo2_after.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Verification
torch.manual_seed(42)
encoder = TransformerEncoder(
    vocab_size=1000, d_model=64, num_heads=4,
    d_ff=256, num_layers=3, max_len=128
)

test_tokens = torch.randint(0, 1000, (2, 20))
out_enc, attn_list = encoder(test_tokens)

assert out_enc.shape == (2, 20, 64), f"Expected (2, 20, 64), got {out_enc.shape}"
assert len(attn_list) == 3, f"Expected 3 layers of attention, got {len(attn_list)}"
assert attn_list[0].shape == (2, 4, 20, 20), f"Wrong attention shape: {attn_list[0].shape}"
print("All assertions passed! Your Transformer encoder is correct!")

In [ ]:
#@title 🎧 Before You Start: Todo2 Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_18_todo2_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### TODO 2: Implement Pre-Norm vs Post-Norm

In [ ]:
#@title 🎧 Before You Start: Todo2 Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_19_todo2_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
class TransformerBlockPreNorm(nn.Module):
    """
    Transformer block with Pre-LayerNorm (used in GPT-2, LLaMA).

    The difference from our original block:
    - Original (Post-Norm):  x = LayerNorm(x + Attention(x))
    - Pre-Norm:              x = x + Attention(LayerNorm(x))

    Pre-Norm tends to train more stably, especially in deep models.
    """
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        self.attention = MultiHeadSelfAttention(d_model, num_heads, dropout)
        self.ffn = FeedForwardNetwork(d_model, d_ff, dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)

    def forward(self, x, mask=None):
        # ============ TODO ============
        # Implement PRE-NORM Transformer block:
        # Sub-layer 1: x = x + Attention(LayerNorm(x))
        # Sub-layer 2: x = x + FFN(LayerNorm(x))
        #
        # Note: LayerNorm is applied BEFORE the sub-layer, not after
        # ==============================

        # YOUR CODE HERE

        return x, attn_weights

In [ ]:
# Verification
torch.manual_seed(42)
block_pre = TransformerBlockPreNorm(d_model=32, num_heads=4, d_ff=128)
x_pre = torch.randn(1, 6, 32)
out_pre, _ = block_pre(x_pre)
assert out_pre.shape == x_pre.shape, f"Shape mismatch: {out_pre.shape}"
print("Pre-Norm block assertions passed!")

In [ ]:
#@title 🎧 Transition: Putting It Together Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_21_putting_it_together_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 6. Putting It All Together

In [ ]:
#@title 🎧 Code Walkthrough: Classifier Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_22_classifier_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
class TransformerClassifier(nn.Module):
    """
    Complete Transformer-based text classifier.
    Embedding + PE + N Transformer blocks + Mean Pooling + Classification Head
    """
    def __init__(self, vocab_size, d_model, num_heads, d_ff, num_layers, num_classes, max_len=512, dropout=0.1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoding = SinusoidalPositionalEncoding(d_model, max_len)
        self.blocks = nn.ModuleList([
            TransformerBlock(d_model, num_heads, d_ff, dropout)
            for _ in range(num_layers)
        ])
        self.classifier = nn.Linear(d_model, num_classes)
        self.dropout = nn.Dropout(dropout)
        self.d_model = d_model

    def forward(self, x, mask=None):
        # Embed + PE
        x = self.embedding(x) * math.sqrt(self.d_model)
        x = self.pos_encoding(x)
        x = self.dropout(x)

        # Transformer blocks
        all_attn = []
        for block in self.blocks:
            x, attn = block(x, mask)
            all_attn.append(attn)

        # Mean pooling over sequence dimension
        x_pooled = x.mean(dim=1)  # (batch, d_model)

        # Classification
        logits = self.classifier(x_pooled)  # (batch, num_classes)

        return logits, all_attn

# Create the model
torch.manual_seed(42)
model = TransformerClassifier(
    vocab_size=200, d_model=32, num_heads=4,
    d_ff=128, num_layers=2, num_classes=2
)

total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params:,}")

In [ ]:
#@title 🎧 Transition: Training Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_23_training_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 7. Training and Results

Let us train on a synthetic sentiment classification task.

In [ ]:
#@title 🎧 Code Walkthrough: Synthetic Data Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_24_synthetic_data_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Create synthetic data: simple patterns that require attention
torch.manual_seed(42)
np.random.seed(42)

vocab_size = 200
seq_len = 20
n_samples = 500

# Pattern: sequences with "positive" tokens (100-149) vs "negative" tokens (150-199)
# Label = majority class in the sequence
X_data = []
y_data = []

for _ in range(n_samples):
    # Choose label
    label = np.random.randint(2)

    if label == 1:  # Positive
        # Mostly tokens from 100-149 with some noise
        seq = np.random.choice(range(100, 150), size=seq_len-4)
        noise = np.random.choice(range(1, 100), size=4)
        seq = np.concatenate([seq, noise])
    else:  # Negative
        # Mostly tokens from 150-199 with some noise
        seq = np.random.choice(range(150, 200), size=seq_len-4)
        noise = np.random.choice(range(1, 100), size=4)
        seq = np.concatenate([seq, noise])

    np.random.shuffle(seq)
    X_data.append(seq)
    y_data.append(label)

X_tensor = torch.tensor(np.array(X_data), dtype=torch.long)
y_tensor = torch.tensor(y_data, dtype=torch.long)

# Split into train/test
train_X, test_X = X_tensor[:400], X_tensor[400:]
train_y, test_y = y_tensor[:400], y_tensor[400:]

train_loader = DataLoader(TensorDataset(train_X, train_y), batch_size=32, shuffle=True)
test_loader = DataLoader(TensorDataset(test_X, test_y), batch_size=32)

print(f"Train: {len(train_X)} samples, Test: {len(test_X)} samples")
print(f"Sequence length: {seq_len}, Vocab size: {vocab_size}")

In [ ]:
#@title 🎧 Code Walkthrough: Training Loop Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_25_training_loop_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Training loop
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
train_losses = []
train_accs = []
test_accs = []

for epoch in range(30):
    model.train()
    epoch_loss = 0
    correct = 0
    total = 0

    for batch_X, batch_y in train_loader:
        logits, _ = model(batch_X)
        loss = F.cross_entropy(logits, batch_y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item() * batch_X.size(0)
        correct += (logits.argmax(dim=1) == batch_y).sum().item()
        total += batch_X.size(0)

    train_losses.append(epoch_loss / total)
    train_accs.append(correct / total)

    # Test accuracy
    model.eval()
    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            logits, _ = model(batch_X)
            correct_test += (logits.argmax(dim=1) == batch_y).sum().item()
            total_test += batch_X.size(0)
    test_accs.append(correct_test / total_test)

    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1:3d}: loss={train_losses[-1]:.4f}  "
              f"train_acc={train_accs[-1]:.3f}  test_acc={test_accs[-1]:.3f}")

In [ ]:
#@title 🎧 What to Look For: Training Plot Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_26_training_plot_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Plot training curves
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(train_losses, color='#e74c3c', linewidth=2)
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12)
ax1.set_title('Training Loss', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)

ax2.plot(train_accs, label='Train', color='#3498db', linewidth=2)
ax2.plot(test_accs, label='Test', color='#2ecc71', linewidth=2, linestyle='--')
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Accuracy', fontsize=12)
ax2.set_title('Accuracy', fontsize=14, fontweight='bold')
ax2.legend(fontsize=11)
ax2.set_ylim(0.4, 1.05)
ax2.grid(True, alpha=0.3)

plt.suptitle('Transformer Classifier Training', fontsize=15, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 Transition: Final Output Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_27_final_output_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 8. Final Output

In [ ]:
#@title 🎧 What to Look For: Attention Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_28_attention_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Visualize attention patterns across all layers and heads
model.eval()
sample_idx = 0
sample_X = test_X[sample_idx:sample_idx+1]
sample_y = test_y[sample_idx].item()

with torch.no_grad():
    logits, all_attn = model(sample_X)
    pred = logits.argmax(dim=1).item()

num_layers = len(all_attn)
num_heads = all_attn[0].shape[1]

fig, axes = plt.subplots(num_layers, num_heads, figsize=(4*num_heads, 4*num_layers))

for layer_idx in range(num_layers):
    for head_idx in range(num_heads):
        ax = axes[layer_idx][head_idx] if num_layers > 1 else axes[head_idx]
        w = all_attn[layer_idx][0, head_idx].numpy()
        im = ax.imshow(w, cmap='Blues', vmin=0, vmax=w.max())
        if layer_idx == 0:
            ax.set_title(f'Head {head_idx+1}', fontsize=11, fontweight='bold')
        if head_idx == 0:
            ax.set_ylabel(f'Layer {layer_idx+1}', fontsize=11, fontweight='bold')
        ax.set_xticks([])
        ax.set_yticks([])

label_names = ['Negative', 'Positive']
plt.suptitle(f'Attention Patterns — Predicted: {label_names[pred]}, '
             f'True: {label_names[sample_y]}',
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 Code Walkthrough: Architecture Summary
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_29_architecture_summary.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Final summary: model architecture diagram
print("=" * 60)
print("TRANSFORMER CLASSIFIER ARCHITECTURE")
print("=" * 60)
print(f"  Vocabulary size:   {vocab_size}")
print(f"  Model dimension:   {model.d_model}")
print(f"  Attention heads:   {num_heads}")
print(f"  FFN hidden dim:    128")
print(f"  Transformer layers: {num_layers}")
print(f"  Total parameters:  {total_params:,}")
print(f"  Max sequence len:  512")
print()
print("  Architecture:")
print("  Token IDs -> Embedding -> + Positional Encoding")
print("    -> [Transformer Block x N]")
print("        -> Multi-Head Attention + Add & Norm")
print("        -> Feed-Forward Network + Add & Norm")
print("    -> Mean Pooling -> Linear -> Class Logits")
print()
print(f"  Final test accuracy: {test_accs[-1]*100:.1f}%")
print("=" * 60)

print("\nCongratulations! You have built a complete Transformer from scratch!")
print("From QKV projections to attention to the full encoder block,")
print("you now understand every single matrix multiplication in the architecture.")

In [ ]:
#@title 🎧 Wrap-Up: Closing
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_30_closing.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 9. Reflection and Next Steps

### Reflection Questions
1. What happens when you increase the number of layers? Does the model become harder to train? How do residual connections help?
2. Why do we scale the embedding by $\sqrt{d_{\text{model}}}$ before adding positional encoding? (Hint: think about the relative magnitudes of the embedding and positional vectors.)
3. The FFN uses a 4x expansion ratio ($d_{ff} = 4 \times d_{\text{model}}$). Why not 2x or 8x? Research suggests this is a good trade-off between expressiveness and compute — but what would happen at the extremes?

### Optional Challenges
1. Implement a **Transformer decoder** with causal masking and cross-attention. This is the architecture behind GPT-style models.
2. Replace the synthetic data with real IMDB movie review data (available via `torchtext` or HuggingFace datasets). Train the model on actual sentiment classification.
3. Add **learning rate warmup** (linearly increasing the LR for the first few epochs). This is standard practice for training Transformers and can significantly improve convergence.